In [1]:
#!pip install lightgbm==3.3.2

In [ ]:
#!pip install xgboost==1.5.0

In [ ]:
#!pip install hyperopt

In [ ]:
# 필요 모듈 로딩, 학습 데이터 dataframe 로딩

# mission : 데이터셋의 shape를 확인하고, 데이터셋을 3행만 출력해주세요.

In [ ]:
# mission : 전처리를 위해 데이터셋을 살펴보세요 
''' hint : 피쳐의 타입과 null 값을 확인하는 함수와 
    피처들의 수학적 통계값을 확인하는 함수를 활용하세요 '''

In [ ]:
# Target 속성의 값 분포 확인하기
# mission :  전체 데이터 셋에서 만족, 불만족 각각의 개수와 비율을 파악해봅시다

''' hint : 다음 두 함수를 활용해요 ! 
    - value_counts(): 고유 값의 빈도를 구함.
    - count(): 결측치가 아닌 값의 총 개수를 구함. '''

# 전처리
##### 어떤 전처리를 해야할까요 ?  위의 데이터 살펴보기 과정에서 어떤 문제점을 발견했나요?

In [3]:
### sweetviz 사용해보기 !

''' import sweetviz as sv
import pandas as pd

# 데이터셋 로드 (예: pandas DataFrame)
df = ____

# Sweetviz 보고서 생성
report = sv.analyze(df, pairwise_analysis='off')
# 보고서 HTML 파일로 저장
report.show_html('sweetviz_report.html') '''

In [ ]:
# 피처 세트와 레이블 세트 분리

### hint : X_features와 y_labels를 분리해주세요 ~

In [ ]:
# 학습/테스트 데이터 세트 분리

### hint :  sklearn 패키지를 이용해요 :)

In [ ]:
# mission : 
# 학습 세트 레이블 값 분포 비율 , 테스트 세트 레이블 값 분포 비율을 출력해볼까요?

### hint : 아까 사용했던 value_counts, count 함수를 활용해보아요 ~

## 본격 모델링을 시작합니다. 
### 배운 내용을 떠올리며, 빈칸을 채워보아요 :)

# XGBoost

In [ ]:
# XGBoost 조기 중단의 검증 데이터 세트 위해 X_train, y_train 다시 학습/검증 분리
X_tr, X_val, y_tr, y_val = ___________(X_train, y_train, test_size=0.3, random_state=0)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# XGBClassifier 학습 수행 !
# n_estimators는 500으로, random_state는 예제 수행 시마다 동일 예측 결과를 위해 156이라는 특정 값으로 설정
xgb_clf = ____________(n_estimators=___, learning_rate=0.05, random_state=___)

# 성능 평가 지표를 auc로, 조기 중단 파라미터 100 설정, 학습 수행
xgb_clf.___(X_tr, y_tr, ______________=___, eval_metric="___", eval_set=[(X_tr, y_tr), (X_val, y_val)])

# ROC AUC 점수 계산
xgb_roc_score = ___________(y_test, xgb_clf._____________(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
# HyperOpt로 하이퍼 파라미터 튜닝
from hyperopt import hp

# 입력값, 검색 공간 설정
# max_depth는 5에서 15까지 1간격, min_child_weight는 1에서 6까지 1간격
# colsample_bytree는 0.5에서 0.95 사이, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색
xgb_search_space = {'max_depth': hp.quniform('max_depth', _, __, _),
                   'min_child_weight': hp.quniform('min_child_weight', _ ,_ ,_ ),
                   'colsample_bytree': hp.uniform('colsample_bytree', ___, ___),
                   'learning_rate': hp.uniform('learning_rate', ___, ___),}


### mission : quniform, uniform 함수는 뭘까요 ?

In [ ]:
# 목적 함수, 3 Fold 교차 검증 이용
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1*roc_acu 평균 값 반환
def objective_func(search_space):
    xgb_clf = _________(n_estimators=100, max_depth=int(search_space['max_depth']),
                                                           min_child_weight=int(search_space['min_child_weight']),
                                                           colsample_bytree=search_space['colsample_bytree'],
                                                           learning_rate=search_space['learning_rate'])
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = [ ]
    
    # 3개 k-fold 방식 적용
    kf = _____(n_splits=3)
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[_____], y_train.iloc[_____]
        X_val, y_val = X_train.iloc[_____], y_train.iloc[_____]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.___(X_tr, y_tr, early_stopping_rounds=__, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결괏값 저장
        score = ___________(y_val, xgb_clf.predict_proba(X_val)[:, 1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값 반환하되,
    # HyperOpt는 목적함수의 최솟값 위한 입력값 찾으므로 -1 곱
    return -1 * np.mean(roc_auc_list)

In [ ]:
# fmin() 함수 호출, 최적 하이퍼 파라미터 도출
from hyperopt import fmin, tpe, Trials

trials = Trials()

# fmin() 함수 호출 max_evals 지정된 횟수만큰 반복 후 목적함수의 최솟값 가지는 최적 입력값 추출
best = ____(fn=objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수 지정
           trials=trials, rstate=np.random.default_rng(seed=30))

print('best:', best)

In [ ]:
# n_estimators를 500 증가 후 최적으로 찾은 하이퍼 파라미터를 기반으로 학습과 예측 수행
xgb_clf = __________(n_estimators=___, learning_rate=round(best['learning_rate'], 5),
                       max_depth=int(best['max_depth']),
                       min_child_weight=int(best['min_child_weight']),
                       colsample_bytree=round(best['colsample_bytree'], 5))

# evaluation metric을 auc로, early stopping 100으로 설정하고 학습 수행
xgb_clf.___(X_tr, y_tr, early_stopping_rounds=___,
           eval_metric="___", eval_set=[(X_tr, y_tr), (X_val, y_val)])

xgb_roc_score = ___________(y_test, xgb_clf.predict_proba(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

In [ ]:
# 피처 중요도 그래프
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
_________________(xgb_clf, ax=ax, max_num_features=20, height=0.4)

# LightGBM

In [ ]:
# LightGBM으로 학습 수행
from lightgbm import LGBMClassifier

lgbm_clf = ______________(n_estimators=500)

eval_set = [(X_tr, y_tr), (X_val, y_val)]
lgbm_clf.___(X_tr, y_tr, early_stopping_rounds=100, eval_metric="auc", eval_set=eval_set)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

In [ ]:
# HyperOpt로 하이퍼 파라미터 튜닝
lgbm_search_space = {'num_leaves': hp.quniform('num_leaves', 32, 64, 1),
                     'max_depth': hp.quniform('max_depth', 100, 160, 1),
                   'min_child_samples': hp.quniform('min_child_samples', 60, 100, 1),
                   'subsample': hp.uniform('subsample', 0.7, 1),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)}

In [ ]:
# 목적 함수 생성
def objective_func(search_space):
    lgbm_clf = ____________(n_estimators=100, num_leaves=int(search_space['num_leaves']),
                                                           max_depth=int(search_space['max_depth']),
                                                           min_child_samples=int(search_space['min_child_samples']),
                                                           subsample=search_space['subsample'],
                                                           learning_rate=search_space['learning_rate'])
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = [ ]
    
    # 3개 k-fold 방식 적용
    kf = _____(n_splits=_)
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[______], y_train.iloc[______]
        X_val, y_val = X_train.iloc[______], y_train.iloc[______]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 LGBMClassifier 학습 수행
        lgbm_clf.___(X_tr, y_tr, early_stopping_rounds=__, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결괏값 저장
        score = _________(y_val, xgb_clf.predict_proba(X_val)[:, 1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값 반환하되,
    # HyperOpt는 목적함수의 최솟값 위한 입력값 찾으므로 -1 곱
    return -1 * np.mean(roc_auc_list)

In [ ]:
# fmin() 함수 호출, 최적 하이퍼 파라미터 도출
from hyperopt import fmin, tpe, Trials

trials = Trials()

# fmin() 함수 호출 max_evals 지정된 횟수만큰 반복 후 목적함수의 최솟값 가지는 최적 입력값 추출
best = ____(fn=objective_func,
           space=lgbm_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수 지정
           trials=trials, rstate=np.random.default_rng(seed=30))

print('best:', best)

In [ ]:
# 최적 하이퍼 파라미터로 LightGBM 학습, ROC-AUC 평가
lgbm_clf = _______(n_estimators=500, num_leaves=int(best['num_leaves']),
                                            max_depth=int(best['max_depth']),
                                            min_child_samples=int(best['min_child_samples']),
                                            subsample=round(best['subsample'], 5),
                                            learning_rate=round(best['learning_rate'], 5))

# evaluation metric을 auc로, early stopping 100으로 설정하고 학습 수행
lgbm_clf.___(X_tr, y_tr, early_stopping_rounds=___,
           eval_metric="___", eval_set=[(X_tr, y_tr), (X_val, y_val)])

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:, 1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))